In [ ]:
import requests
import csv
from time import sleep

def get_artist_genre_musicbrainz(artist_name):
    base_url = "https://musicbrainz.org/ws/2/artist/"
    headers = {'User-Agent': 'GenreFinder/1.0 (your_email@example.com)'}  # Required!
    params = {
        'query': f'artist:"{artist_name}"',
        'fmt': 'json',
        'limit': 1
    }
    
    try:
        # First try: Exact match
        response = requests.get(base_url, headers=headers, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        # Second try: Fuzzy match if no results
        if not data.get('artists'):
            params['query'] = artist_name  # Remove exact match quotes
            response = requests.get(base_url, headers=headers, params=params, timeout=10)
            data = response.json()
        
        # Extract genres
        if data.get('artists'):
            tags = [tag['name'] for tag in data['artists'][0].get('tags', [])]
            return tags[:3] if tags else ['No genres tagged']
        return ['Artist not found']
    
    except Exception as e:
        return [f'API Error: {str(e)}']

def process_csv(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as infile, \
         open(output_file, 'w', encoding='utf-8', newline='') as outfile:
        
        reader = csv.reader(infile)
        writer = csv.writer(outfile)
        writer.writerow(['Artist', 'Genres'])
        
        for row in reader:
            artist = row[0].strip()
            if artist:
                print(f"Processing: {artist}")
                genres = get_artist_genre_musicbrainz(artist)
                writer.writerow([artist, ', '.join(genres)])
                sleep(1.2)  # Respect rate limits

# Usage
process_csv('/Users/oluayinde/Downloads/youtube_data - artists.csv', '/Users/oluayinde/Downloads/artists_genres_musicbrainz.csv')